In [ ]:
import math
from matplotlib import pyplot as plt
import numpy as np
import numpy.random as nprand
import pandas as pd
import random
import scipy.stats as spstats
%matplotlib inline

from loomio import *
from socialchoice import *
from netdelib import *

In [ ]:
df_score, proposal_map, proposal_rev_map = load_loomio_score('results/results_2_3.tsv')
stage_profiles = [
    Profile.from_score(df_score[df_score.stage == stage])
    for stage in sorted(set(df_score.stage))]

Methods = [Condorcet, Majority, Borda, Tideman]
print('stage\t' + '\t'.join([M.__name__ for M in Methods]))
for stage, profile in enumerate(stage_profiles):
    winners = [repr(M(profile).social_choice()) for M in Methods]
    print(str(stage) + '\t' + '\t'.join(winners))


## Group-Level Results

In [ ]:
df_score, proposal_map, proposal_rev_map = load_loomio_score('results/results_2_3.tsv')
df_score.groupby(['treatment', 'stage']).count()

In [ ]:
treatments = {
    1: "Single Group",
    2: "Random Pod",
}

for treatment in [1, 2]:
    print("Treatment: {}".format(treatments[treatment]))
    stage_profiles = [
        Profile.from_score(df_score[(df_score.stage == stage) & (df_score.treatment == treatment)])
        for stage in sorted(set(df_score.stage))]

    Methods = [Condorcet, Majority, Borda, Tideman]
    print('stage\t' + '\t'.join([M.__name__ for M in Methods]))
    for stage, profile in enumerate(stage_profiles):
        winners = [repr(M(profile).social_choice()) for M in Methods]
        print(str(stage) + '\t' + '\t'.join(winners))
    print("")


In [ ]:
for treatment in [1, 2]:
    stage_profiles = [
        Profile.from_score(df_score[(df_score.stage == stage) & (df_score.treatment == treatment)])
        for stage in sorted(set(df_score.stage))]
    for stage, profile in enumerate(stage_profiles):
        for pref in KemenyYoung(profile).social_preference_set():
            print("{}\t".format(stage), pref)
    print("")


### Group-Level Agreement

In [ ]:
df_score, proposal_map, proposal_rev_map = load_loomio_score('results/results_2_3.tsv')
netdelib = NetDelibAgreement(df_score)

In [ ]:
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
netdelib.plot_kendall()
plt.subplot(1,2,2)
netdelib.plot_swap()
plt.tight_layout()

## Group-Level Results (Remove Attrition)

In [ ]:
df_score, proposal_map, proposal_rev_map = load_loomio_score('results/results_2_3.tsv')
df_score = remove_attrition(df_score)

treatments = {
    1: "Single Group",
    2: "Random Pod",
}

for treatment in [1, 2]:
    print("Treatment: {}".format(treatments[treatment]))
    stage_profiles = [
        Profile.from_score(df_score[(df_score.stage == stage) & (df_score.treatment == treatment)])
        for stage in sorted(set(df_score.stage))]

    Methods = [Majority, Borda, Tideman]
    print('stage\t' + '\t'.join([M.__name__ for M in Methods]))
    for stage, profile in enumerate(stage_profiles):
        winners = [repr(M(profile).social_choice()) for M in Methods]
        print(str(stage) + '\t' + '\t'.join(winners))
    print("")


### Group-Level Agreement (Remove Attrition)

In [ ]:
netdelib = NetDelibAgreement(df_score)

In [ ]:
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
netdelib.plot_kendall()
plt.subplot(1,2,2)
netdelib.plot_swap()
plt.tight_layout()

## Group-Level Results (Forward-Fill Attrition)

In [ ]:
df_score, proposal_map, proposal_rev_map = load_loomio_score('results/results_2_3.tsv')
df_score = fill_attrition(df_score)

treatments = {
    1: "Single Group",
    2: "Random Pod",
}

for treatment in [1, 2]:
    print("Treatment: {}".format(treatments[treatment]))
    stage_profiles = [
        Profile.from_score(df_score[(df_score.stage == stage) & (df_score.treatment == treatment)])
        for stage in sorted(set(df_score.stage))]

    Methods = [Majority, Borda, Tideman]
    print('stage\t' + '\t'.join([M.__name__ for M in Methods]))
    for stage, profile in enumerate(stage_profiles):
        winners = [repr(M(profile).social_choice()) for M in Methods]
        print(str(stage) + '\t' + '\t'.join(winners))
    print("")


### Group-Level Agreement (Forward-Fill Attrition)

In [ ]:
netdelib = NetDelibAgreement(df_score)

In [ ]:
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
netdelib.plot_kendall()
plt.subplot(1,2,2)
netdelib.plot_swap()
plt.tight_layout()